# Portfolio Backtest-er
Enter your desired securities and their optimal weightings. The final cell backtests each weight scheme for easy comparison.

In [1]:
import yfinance as yf
import numpy as np
import pandas as pd

In [2]:
'''Enter desired security symbols'''
tickers = ['nvda', 'f', 'gm', 'aapl']
tickers.sort()
close = pd.DataFrame(columns=tickers)

'''Dict for managing all ticker objects'''
full = {}
for ticker in tickers:
    thing = yf.Ticker(ticker)
    full[ticker] = thing
    
    '''Retrieve and clean historical data'''
    data = thing.history(period='3y', interval='1d')
    close[ticker] = data.loc[:, 'Close']
    
rtns = close.pct_change()
n = len(close.columns)
print(full)

{'aapl': yfinance.Ticker object <AAPL>, 'f': yfinance.Ticker object <F>, 'gm': yfinance.Ticker object <GM>, 'nvda': yfinance.Ticker object <NVDA>}


In [3]:
'''Enter the optimal portfolio weights'''
opt_w = [0.42133033344520693, 0.31090500753766304, -0.16441472892500425, 0.43217938794213423]
'''Equal weighted pofo'''
eq_w = [0.2, 0.2, 0.2, 0.2, 0.2]

In [4]:
'''Price-weighted portfolio'''
p_w = []

tot_p = 0
for i in range(n):
    tot_p += close.iloc[-1, i]
    
for i in range(n):
    weight = close.iloc[-1, i] / tot_p
    p_w.append(weight)

In [8]:
'''Value-weighted portfolio'''
m_w = []

tot_v = 0
v = []
main = list(full)
for i in range(n):
    idx = main[i]
    value = full[idx].info['sharesOutstanding'] * close.iloc[-1, i]
    v.append(value)
    tot_v += value
    
for i in range(n):
    weight = v[i] / tot_v
    m_w.append(weight)

In [6]:
'''Define function for vectorized backtesting'''
def backtest(x, df):
    tot = []
    for i in range(n):
        col = df.iloc[:, i] * x[i]
        tot.append(col.sum())
    return sum(tot)

In [7]:
'''Backtest each scheme and display results'''
optimal = round(backtest(opt_w, rtns) * 100, 2)
equal = round(backtest(eq_w, rtns) * 100, 2)
price = round(backtest(p_w, rtns) * 100, 2)
market_value = round(backtest(m_w, rtns) * 100, 2)

print('Returns for following schemes:')
print(
    f'Optimal: {optimal}%\nEqual: {equal}%\nPrice: {price}%\nMarket Value: {market_value}%'
)

Returns for following schemes:
Optimal: 189.74%
Equal: 116.97%
Price: 196.65%
Market Value: 168.48%
